This notebook is to clean adata, add the counts in the same barcode

In [14]:
import scanpy as sc
import os
import re
import scipy as scp
import pandas as pd
import scvelo as scv
import anndata as ad

In [2]:
adata = sc.read_h5ad('processed_adata.h5ad')
adata

AnnData object with n_obs × n_vars = 223413 × 22555 
    obs: 'folder', 'n_genes'
    var: 'n_cells'
    layers: 'spliced', 'unspliced'

In [3]:
phenoDat = pd.read_csv('phenoSCCall.csv')
phenoDat.drop('Unnamed: 0',axis=1, inplace=True)
phenoDat.head()

,cell,treatment,replicate,barcodes
0,SCC1,CTX,R1,AAACCTGAGTTGTCGT
1,SCC1,CTX,R1,AAACCTGCACAGCGTC
2,SCC1,CTX,R1,AAACCTGGTACTTCTT
3,SCC1,CTX,R1,AAAGATGGTGTGAATA
4,SCC1,CTX,R1,AAAGCAAGTAGCTGCC


Clean the adata to add counts from same cell 

In [4]:
adataX = pd.DataFrame(adata.X.toarray(),index=adata.obs.index,columns=adata.var.index)
adataX.head()

index,ENSG00000001460.18,ENSG00000001461.17,ENSG00000010072.16,ENSG00000009780.15,ENSG00000048707.15,ENSG00000059588.10,ENSG00000041988.15,ENSG00000049245.13,ENSG00000052723.11,ENSG00000058799.15,...,ENSG00000161835.11,ENSG00000173809.18,ENSG00000237126.8,ENSG00000236699.9,ENSG00000242252.2,ENSG00000242461.1,ENSG00000231364.3,ENSG00000288023.1,ENSG00000211445.12,ENSG00000267547.1
index,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCGATTCT-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGCTTTGGT-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGGCCCTCA-0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGTAGCCGA-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGGTAGCGCTC-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
adataSp = pd.DataFrame(adata.layers['spliced'].toarray(),index=adata.obs.index,columns=adata.var.index)
adataSp.head()

index,ENSG00000001460.18,ENSG00000001461.17,ENSG00000010072.16,ENSG00000009780.15,ENSG00000048707.15,ENSG00000059588.10,ENSG00000041988.15,ENSG00000049245.13,ENSG00000052723.11,ENSG00000058799.15,...,ENSG00000161835.11,ENSG00000173809.18,ENSG00000237126.8,ENSG00000236699.9,ENSG00000242252.2,ENSG00000242461.1,ENSG00000231364.3,ENSG00000288023.1,ENSG00000211445.12,ENSG00000267547.1
index,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCGATTCT-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGCTTTGGT-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGGCCCTCA-0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGTAGCCGA-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGGTAGCGCTC-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
adataUsp = pd.DataFrame(adata.layers['spliced'].toarray(),index=adata.obs.index,columns=adata.var.index)
adataUsp.head()

index,ENSG00000001460.18,ENSG00000001461.17,ENSG00000010072.16,ENSG00000009780.15,ENSG00000048707.15,ENSG00000059588.10,ENSG00000041988.15,ENSG00000049245.13,ENSG00000052723.11,ENSG00000058799.15,...,ENSG00000161835.11,ENSG00000173809.18,ENSG00000237126.8,ENSG00000236699.9,ENSG00000242252.2,ENSG00000242461.1,ENSG00000231364.3,ENSG00000288023.1,ENSG00000211445.12,ENSG00000267547.1
index,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCGATTCT-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGCTTTGGT-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGGCCCTCA-0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGTAGCCGA-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGGTAGCGCTC-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
adata.var.head()

,n_cells
index,
ENSG00000001460.18,7252.0
ENSG00000001461.17,31838.0
ENSG00000010072.16,22712.0
ENSG00000009780.15,15246.0
ENSG00000048707.15,11175.0


In [8]:
adata.obs.head()

,folder,n_genes
index,,
AAACCTGAGCGATTCT-0,S4_L007,2596
AAACCTGAGCTTTGGT-0,S4_L007,329
AAACCTGAGGCCCTCA-0,S4_L007,2178
AAACCTGAGTAGCCGA-0,S4_L007,253
AAACCTGGTAGCGCTC-0,S4_L007,420


Sequence of cleaning the adata:
    1. Add data, spliced and unspliced,
    2. Remove redundant columns
    3. Recompute obs and var

In [9]:
d = {}
drops = []
for i,x in enumerate(adata.obs.index):
    if x[:16] not in d:
        d[x[:16]] = i
    else:
        j = d[x[:16]]
        adataX.iloc[j,:] += adataX.iloc[j,:]
        adataSp.iloc[j,:] += adataSp.iloc[j,:] 
        adataUsp.iloc[j,:] += adataUsp.iloc[j,:] 
        drops.append(x)

In [10]:
adataX.drop(index=drops,inplace=True)
adataSp.drop(index=drops,inplace=True)
adataUsp.drop(index=drops,inplace=True)

In [11]:
del d
del drops

In [12]:
adataX = scp.sparse.csr_matrix(adataX.values)
adataSp = scp.sparse.csr_matrix(adataSp.values)
adataUsp = scp.sparse.csr_matrix(adataUsp.values)

In [17]:
adat = ad.AnnData(X=adataX)
adat.layers['spliced'] = adataSp
adat.layers['unspliced'] = adataUsp

In [18]:
adat

AnnData object with n_obs × n_vars = 33209 × 22555 
    layers: 'spliced', 'unspliced'

In [21]:
sc.pp.filter_cells(adat,min_genes=200)
sc.pp.filter_genes(adat,min_cells=3)
adat

AnnData object with n_obs × n_vars = 33189 × 19804 
    obs: 'n_genes'
    var: 'n_cells'
    layers: 'spliced', 'unspliced'

In [23]:
adat.obs.head()

,n_genes
0,2592
1,328
2,2177
3,253
4,420
